# **Network Data Source**: Host <-> Network Join
------------------

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_zeek = pd.read_json("df_zeek_labelled3.json", lines = True)
df_zeek.replace('nan', np.nan, inplace=True)
print(type(df_zeek))
print(df_zeek.shape)
print(df_zeek.EventID.value_counts())
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>
(2248, 158)
5156.0    67
3.0       67
Name: EventID, dtype: int64


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names,Source_IP,Source_Port_x,Destination_IP,Destination_Port,ts_pd,EventID,TP_Binary,TP_Label,@timestamp,Source_Port_y
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,54243.0,10.0.0.4,53.0,2020-04-30 00:06:39.824369907+00:00,NaN,False,0,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,56880.0,10.0.0.4,445.0,2020-04-30 00:06:39.827696085+00:00,NaN,False,0,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,65108.0,10.0.0.4,53.0,2020-04-30 00:07:59.953876019+00:00,

In [3]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)
print(type(df_label))
print(df_label.shape)
print(df_label.EventID.value_counts())
df_label.head(1)

<class 'pandas.core.frame.DataFrame'>
(198760, 390)
12       61158
10       39286
7        20259
13       17542
4658     10973
         ...  
172          1
46           1
50103        1
306          1
41           1
Name: EventID, Length: 165, dtype: int64


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

In [4]:
df_label.TP_Binary.value_counts()

False    195322
True       3438
Name: TP_Binary, dtype: int64

In [5]:
# Propagate network data label to all connections within a uid?

In [6]:
# Looking for False Positives with TP_Binary=True and ts outside of true positive window.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_zeek.groupby(['ts', 'EventID', 'TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'])['@timestamp', 'EventID', 'TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


@timestamp  \
ts           EventID TP_Binary uid                Source_IP Destination_IP Destination_Port               
1.588205e+09 3.0     True      CUFN2104UUWgAE48   10.0.1.4  192.168.0.5    1234.0                     1   
             5156.0  True      CUFN2104UUWgAE48   10.0.1.4  192.168.0.5    1234.0                     1   
1.588206e+09 3.0     True      CnfxOnz9EEuMywvih  10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CnfxOnz9EEuMywvih  10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CRxmG32y1yTI2Odtt9 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CRxmG32y1yTI2Odtt9 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      C4r8wf2BmPKRfanAF6 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      C4r8wf2BmPKRfanAF6 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CFmtFn4oHdLD2jVYD2 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CFmtFn4oHdLD2jVYD2 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      C6aOhX3MGEBfgSoXW6 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      C6aOhX3MGEBfgSoXW6 10.0.1.4  10.0.0.4       389.0                      2   
1.588207e+09 3.0     True      CzJQsD4vDzP1O4LDK1 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CzJQsD4vDzP1O4LDK1 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      C0vu7WGlpKLjL8Qhc  10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      C0vu7WGlpKLjL8Qhc  10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CgFqEH2AArPQ4DLOYf 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CgFqEH2AArPQ4DLOYf 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CGNgVO1OlaEMzjq5A3 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CGNgVO1OlaEMzjq5A3 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CjqEFp2wB12h7DEGt2 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CjqEFp2wB12h7DEGt2 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CWHEjpiltZznVrMaa  10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CWHEjpiltZznVrMaa  10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CRTaeg2G66cIB9TRm6 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CRTaeg2G66cIB9TRm6 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True  

In [7]:
# Group by ts vs TP_Binary shows if label accurately captures the scope of the connection. 
# No need to propagate label using uid. 1 to 1.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_zeek.groupby(['uid', 'TP_Binary'])['@timestamp', 'EventID', 'TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,,@timestamp,EventID,TP_Binary,uid,Source_IP,Destination_IP,Destination_Port
uid,TP_Binary,,,,,,,
C03eJ14HreyLlDSYXb,False,0,0,1,1,1,1,1
C0Ah071D3JnoodwJO2,False,0,0,2,2,2,2,2
C0LmMn33GTjEqE0YKc,False,0,0,3,3,3,3,3
C0SIhd2lBq9aEbOI1d,False,0,0,2,2,2,2,2
C0XYv11LLkSOaKmcOa,True,8,8,8,8,8,8,8
C0ch1M1Oka8xzF0cR4,False,0,0,2,2,2,2,2
C0texO2ImscKqridG4,False,0,0,2,2,2,2,2
C0vu7WGlpKLjL8Qhc,True,4,4,4,4,4,4,4
C1LaI03eBfyLTLual1,False,0,0,1,1,1,1,1


In [8]:
# Left join host<->network on @timestamp, EventID

In [9]:
df_zeek.TP_Binary.value_counts()

False    2114
True      134
Name: TP_Binary, dtype: int64

In [10]:
# I can test the certainty of the TP_Binary label on the JOIN. 
# See the diff between TP_Binary_x and TP_Binary_y! ID False Positives.
# See if the host->network label prop based on ['Destination_IP', 'Source_IP', 'Destination_Port'] is legit!

In [11]:
df_label.shape

(198760, 390)

In [12]:
df_zeek.shape

(2248, 158)

In [13]:
# Join Host<->Network based on ['EventID', '@timestamp']
# Worked! Trying outer
#df_label = pd.merge(
#    left = df_label,
#    right = df_zeek,
#    how = 'left',
#    on = ['EventID', '@timestamp']
#)

In [14]:
#df_label.shape

In [15]:
# Join Host<->Network based on 
df_label = pd.merge(
    left = df_label,
    right = df_zeek,
    how = 'outer',
    on = ['@timestamp', 'EventID']
)

In [16]:
df_label.shape

(201048, 546)

In [17]:
df_label.head(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host_x,port,tags_x,TP_Label_x,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Group,IdleImplementation,IdleStateCount,MaximumPerformancePercent,MinimumPerformancePercent,MinimumThrottlePercent,NominalFreq

In [18]:
df_label.TP_Binary_x.value_counts()

False    195354
True       3580
Name: TP_Binary_x, dtype: int64

In [19]:
df_label.TP_Binary_y.value_counts()

False    2114
True      196
Name: TP_Binary_y, dtype: int64

In [27]:
# Looking for differences between TP_Label in the Host Data vs the Network Data.
#     -> So 12 hits on Host Data labelled benign that was labelled malicious in the Network Data.
#     -> Could still be accurate label, revealing a shortcoming in the original host SQL labelling technique.
#     -> the 24 hits on TP_Label_x == NaN and TP_Label_y == 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.groupby(['TP_Label_x', 'TP_Binary_x', 'TP_Label_y', 'TP_Binary_y'])['@timestamp', 'EventID'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


@timestamp  \
TP_Label_x                        TP_Binary_x TP_Label_y                        TP_Binary_y               
0                                 False       0, 8.A.1. Remote System Discovery True                 12   
                                              0, 8.A.2. Remote System Discovery True                 24   
0, 1.A.3. Uncommonly Used Port    True        0, 1.A.3. Uncommonly Used Port    True                  2   
0, 3.B.3. Commonly Used Port      True        0, 3.B.3. Commonly Used Port      True                 30   
0, 8.A.1. Remote System Discovery True        0, 8.A.1. Remote System Discovery True                 48   
0, 8.A.2. Remote System Discovery True        0, 8.A.2. Remote System Discovery True                 80   

                                                                                             EventID  
TP_Label_x                        TP_Binary_x TP_Label_y                        TP_Binary_y           
0                                 False       0, 8.A.1. Remote System Discovery True              12  
                                              0, 8.A.2. Remote System Discovery True              24  
0, 1.A.3. Uncommonly Used Port    True        0, 1.A.3. Uncommonly Used Port    True               2  
0, 3.B.3. Commonly Used Port      True        0, 3.B.3. Commonly Used Port      True              30  
0, 8.A.1. Remote System Discovery True        0, 8.A.1. Remote System Discovery True              48  
0, 8.A.2. Remote System Discovery True        0, 8.A.2. Remote System Discovery True              80

In [43]:
# Dive into the potential false positives.
#     -> Looks like the network label picked up the:
#           -> 10.0.1.4 -> 10.0.0.4 Port 88 kerberos auth attempts (20x)  -> could go either way tp/fp
#           -> 10.0.1.4 -> 10.0.1.6 Port 5985 WinRM(PowerShell Remoting) (8x) -> maps to emulation plan. workstation->workstation WinRM is generally tp malicious
#           -> 10.0.1.4 -> 10.0.0.4 Port 53 DNS (8x) -> likely FP. DNS between a domain workstation and it's domain controller is normal.
#     -> I'll just treat TP_Label_x or TP_Binary_x as the correct label going forward.
df_label.loc[(df_label.TP_Label_x == '0') & (df_label.TP_Binary_y == 'True')].groupby(['Message'])['EventID', 'Message'].count()

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


,EventID,Message
Message,,
Network connection detected:\r\nRuleName: -\r\nUtcTime: 2020-05-02 03:09:04.485\r\nProcessGuid: {47ab858c-cad9-5eac-0c00-000000000400}\r\nProcessId: 736\r\nImage: C:\Windows\System32\lsass.exe\r\nUser: NT AUTHORITY\SYSTEM\r\nProtocol: tcp\r\nInitiated: true\r\nSourceIsIpv6: false\r\nSourceIp: 10.0.1.4\r\nSourceHostname: -\r\nSourcePort: 59956\r\nSourcePortName: -\r\nDestinationIsIpv6: false\r\nDestinationIp: 10.0.0.4\r\nDestinationHostname: -\r\nDestinationPort: 88\r\nDestinationPortName: -,12,12
Network connection detected:\r\nRuleName: -\r\nUtcTime: 2020-05-02 03:09:23.794\r\nProcessGuid: {47ab858c-cad9-5eac-0c00-000000000400}\r\nProcessId: 736\r\nImage: C:\Windows\System32\lsass.exe\r\nUser: NT AUTHORITY\SYSTEM\r\nProtocol: tcp\r\nInitiated: true\r\nSourceIsIpv6: false\r\nSourceIp: 10.0.1.4\r\nSourceHostname: -\r\nSourcePort: 59958\r\nSourcePortName: -\r\nDestinationIsIpv6: false\r\nDestinationIp: 10.0.0.4\r\nDestinationHostname: -\r\nDestinationPort: 88\r\nDestinationPortName: -,8,8
Network connection detected:\r\nRuleName: -\r\nUtcTime: 2020-05-02 03:09:29.713\r\nProcessGuid: {5aa8ec29-cad1-5eac-0100-000000000400}\r\nProcessId: 4\r\nImage: System\r\nUser: NT AUTHORITY\SYSTEM\r\nProtocol: tcp\r\nInitiated: false\r\nSourceIsIpv6: false\r\nSourceIp: 10.0.1.4\r\nSourceHostname: -\r\nSourcePort: 59962\r\nSourcePortName: -\r\nDestinationIsIpv6: false\r\nDestinationIp: 10.0.1.6\r\nDestinationHostname: -\r\nDestinationPort: 5985\r\nDestinationPortName: -,8,8
The Windows Filtering Platform has permitted a connection.\r\n\r\nApplication Information:\r\n\tProcess ID:\t\t1636\r\n\tApplication Name:\t\device\harddiskvolume2\windows\system32\svchost.exe\r\n\r\nNetwork Information:\r\n\tDirection:\t\tOutbound\r\n\tSource Address:\t\t10.0.1.4\r\n\tSource Port:\t\t59445\r\n\tDestination Address:\t10.0.0.4\r\n\tDestination Port:\t\t53\r\n\tProtocol:\t\t17\r\n\r\nFilter Information:\r\n\tFilter Run-Time ID:\t68659\r\n\tLayer Name:\t\tConnect\r\n\tLayer Run-Time ID:\t48,8,8


In [44]:
# Write out host<->network outer join to disk
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import length, concat ,concat_ws

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.sparkContext.setCheckpointDir("./tmp/checkpoints")
spark.sparkContext.version
spark.conf.set("spark.driver.memory", "3g")
spark.conf.set("spark.executor.memory", "3g")

In [ ]:
#df_labelled = spark.createDataFrame(df_zeek2.astype(str))
df_labelled = spark.createDataFrame(df_label.astype(str))